In [1]:
import pandas as pd

In [2]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../Configuration"

In [3]:
import refinitiv.data as rd

In [4]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x25cd29d8be0 {name='workspace'}>

In [5]:
df_companies = pd.read_csv('Refinitiv_ESG_Universe.csv', delimiter=';')

df_companies.head()

,ISIN,Name
0,DK0010244508,A P MOLLER MAERSK B
1,DE000A1TNNN5,A S CREATION TAPETEN
2,CA0002551095,A&W REVENUE RYLT.INC.FD. UTS.
3,CNE100002RY5,A-LIVING SERVICES 'H'
4,US00181T1079,A-MARK PRECIOUS METALS


In [6]:
df_companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9960 entries, 0 to 9959
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISIN    9960 non-null   object
 1   Name    9960 non-null   object
dtypes: object(2)
memory usage: 155.8+ KB


In [7]:
# example, also works with ISIN instead of BASF Ticker
df = rd.get_data(
    universe=["BASFn.DE", "IBM.N"],
    fields=["TR.HeadquartersCountry", "TR.HeadquartersRegion", "TR.NAICSNationalIndustry", "TR.NAICSIndustryGroup", 
            "TR.NAICSSector", "TR.GICSSubIndustry", "TR.GICSIndustry", "TR.ICBIndustry", "TR.ICBSector", "TR.GICSIndustryGroup", "TR.SICIndustryGroup"
            "TR.GICSSector", "TR.SICIndustry", "TR.TRBCIndustry", "TR.TRBCIndustryGroup", "TR.TRBCBusinessSector", "TR.TRBCEconomicSector",
            "TR.TSE33SectorNameMain", "TR.NAICSSubsector"]
)

df

,Instrument,Country of Headquarters,Region of Headquarters,NAICS National Industry Name,NAICS Industry Group Name,NAICS Sector Name,GICS Sub-Industry Name,GICS Industry Name,ICB Industry name,ICB Sector name,GICS Industry Group Name,SIC Industry Name,TRBC Industry Name,TRBC Industry Group Name,TRBC Business Sector Name,TRBC Economic Sector Name,TSE33 Main sector name,NAICS Subsector Name
0,BASFn.DE,Germany,Europe,Paint and Coating Manufacturing,"Paint, Coating, and Adhesive Manufacturing",Manufacturing,Diversified Chemicals,Chemicals,Basic Materials,Chemicals,Materials,,Diversified Chemicals,Chemicals,Chemicals,Basic Materials,,Chemical Manufacturing
1,IBM.N,United States of America,Americas,Computer Systems Design Services,Computer Systems Design and Related Services,"Professional, Scientific, and Technical Services",IT Consulting & Other Services,IT Services,Technology,Software & Computer Services,Software & Services,,IT Services & Consulting,Software & IT Services,Software & IT Services,Technology,,"Professional, Scientific, and Technical Services"


In [8]:
# Stuff that doesn't change with time (Sector/Industry/Country/Region)
# testing the loop
test_frame = df_companies.iloc[:15, :]

# loop to collect data for all available companies, stored in a dictionary with a df for each isin
comp_data_dict = {}

for isin in test_frame["ISIN"]:
    df = rd.get_data(
        universe=isin,
        fields=["TR.HeadquartersCountry", "TR.HeadquartersRegion", "TR.NAICSNationalIndustry", "TR.NAICSIndustryGroup", 
            "TR.NAICSSector", "TR.GICSSubIndustry", "TR.GICSIndustry", "TR.ICBIndustry", "TR.ICBSector", "TR.GICSIndustryGroup", "TR.SICIndustryGroup"
            "TR.GICSSector", "TR.SICIndustry", "TR.TRBCIndustry", "TR.TRBCIndustryGroup", "TR.TRBCBusinessSector", "TR.TRBCEconomicSector",
            "TR.TSE33SectorNameMain", "TR.NAICSSubsector"]
    )

    comp_data_dict[isin] = df

comp_dfs = []
for key, df in comp_data_dict.items():
    df['key'] = key  # Add a new column containing the key
    comp_dfs.append(df)

# Concatenate the dataframes into a single one
comp_info_df_test = pd.concat(comp_dfs)

# save the new dataframe
comp_info_df_test.to_csv("comp_info_df_test.csv")

comp_info_df_test.head()

,Instrument,Country of Headquarters,Region of Headquarters,NAICS National Industry Name,NAICS Industry Group Name,NAICS Sector Name,GICS Sub-Industry Name,GICS Industry Name,ICB Industry name,ICB Sector name,GICS Industry Group Name,SIC Industry Name,TRBC Industry Name,TRBC Industry Group Name,TRBC Business Sector Name,TRBC Economic Sector Name,TSE33 Main sector name,NAICS Subsector Name,key
0,DK0010244508,Denmark,Europe,Deep Sea Freight Transportation,"Deep Sea, Coastal, and Great Lakes Water Trans...",Transportation and Warehousing,Marine,Marine,Industrials,Industrial Transportation,Transportation,,Marine Freight & Logistics,Freight & Logistics Services,Transportation,Industrials,,Water Transportation,DK0010244508
0,DE000A1TNNN5,Germany,Europe,Paper Bag and Coated and Treated Paper Manufac...,Converted Paper Product Manufacturing,Manufacturing,Home Furnishings,Household Durables,Consumer Discretionary,Household Goods and Home Construction,Consumer Durables & Apparel,,Home Furnishings,Household Goods,Cyclical Consumer Products,Consumer Cyclicals,,Paper Manufacturing,DE000A1TNNN5
0,CA0002551095,Canada,Americas,Full-Service Restaurants,Restaurants and Other Eating Places,Accommodation and Food Services,Restaurants,"Hotels, Restaurants & Leisure",Consumer Discretionary,Travel and Leisure,Consumer Services,,Restaurants & Bars,Hotels & Entertainment Services,Cyclical Consumer Services,Consumer Cyclicals,,Food Services and Drinking Places,CA0002551095
0,CNE100002RY5,China,Asia,Residential Property Managers,Activities Related to Real Estate,Real Estate and Rental and Leasing,Real Estate Operating Companies,Real Estate Management & Development,Real Estate,Real Estate Investment and Services Development,Real Estate,,Real Estate Services,Real Estate Operations,Real Estate,Real Estate,,Real Estate,CNE100002RY5
0,US00181T1079,United States of America,Americas,"Jewelry, Watch, Precious Stone, and Precious M...",Miscellaneous Durable Goods Merchant Wholesalers,Wholesale Trade,Specialized Finance,Diversified Financial Services,Consumer Discretionary,Retailers,Diversified Financials,"Jewelry, Watches, Precious Stones, and Preciou...",Non-Gold Precious Metals & Minerals,Metals & Mining,Mineral Resources,Basic Materials,,"Merchant Wholesalers, Durable Goods",US00181T1079


In [9]:
# Stuff that doesn't change with time (Sector/Industry/Country/Region)

# loop to collect data for all available companies, stored in a dictionary with a df for each isin
comp_data_dict = {}

for isin in df_companies["ISIN"]:
    
    df = rd.get_data(
        universe=isin,
        fields=["TR.HeadquartersCountry", "TR.HeadquartersRegion", "TR.NAICSNationalIndustry", "TR.NAICSIndustryGroup", 
            "TR.NAICSSector", "TR.GICSSubIndustry", "TR.GICSIndustry", "TR.ICBIndustry", "TR.ICBSector", "TR.GICSIndustryGroup", "TR.SICIndustryGroup"
            "TR.GICSSector", "TR.SICIndustry", "TR.TRBCIndustry", "TR.TRBCIndustryGroup", "TR.TRBCBusinessSector", "TR.TRBCEconomicSector",
            "TR.TSE33SectorNameMain", "TR.NAICSSubsector"]
    )

    comp_data_dict[isin] = df

comp_dfs = []
for key, df in comp_data_dict.items():
    df['key'] = key  # Add a new column containing the key
    comp_dfs.append(df)

# Concatenate the dataframes into a single one
comp_info_df = pd.concat(comp_dfs)

# save the new dataframe
comp_info_df.to_csv("comp_info_df.csv")

In [10]:
# testing the loop
test_frame = df_companies.iloc[:15, :]

# loop to collect data for all available companies, stored in a dictionary with a df for each isin
financials_data_dict = {}

for isin in test_frame["ISIN"]:

    df = rd.get_history(
        universe=isin,
        fields=["TR.RevenueMean", "TR.EarningsMean", "TR.F.EmpAvg", "TR.EPSMean", "TR.ROAMean", "TR.ROEMean",
        "TR.Volume", "TR.CompanyMarketCap", "TR.FCFMean", "TR.WACC", "TR.EBITDAMean", "TR.OrganicSalesGrowthMean"],
        interval="1Y",
        start="2000-01-01",
        end="2023-01-01",
    )
    
    df.index = pd.to_datetime(df.index)
    df = df.groupby(df.index.year).first()

    financials_data_dict[isin] = df

# Assume `dfs_dict` is the dictionary of dataframes
dfs = []
for key, df in financials_data_dict.items():
    df['key'] = key  # Add a new column containing the key
    dfs.append(df)

# Concatenate the dataframes into a single one
result = pd.concat(dfs)

result.to_csv('comp_financials_test.csv')

result.head()

,key


In [11]:
# loop to collect data for all available companies, stored in a dictionary with a df for each isin
financials_data_dict = {}

for isin in df_companies["ISIN"]:

    df = rd.get_history(
        universe=isin,
        fields=["TR.RevenueMean", "TR.EarningsMean", "TR.F.EmpAvg", "TR.EPSMean", "TR.ROAMean", "TR.ROEMean",
        "TR.Volume", "TR.CompanyMarketCap", "TR.FCFMean", "TR.WACC", "TR.EBITDAMean", "TR.OrganicSalesGrowthMean"],
        interval="1Y",
        start="2000-01-01",
        end="2023-01-01",
    )
    
    df.index = pd.to_datetime(df.index)
    df = df.groupby(df.index.year).first()

    financials_data_dict[isin] = df

# Assume `dfs_dict` is the dictionary of dataframes
dfs = []
for key, df in financials_data_dict.items():
    df['key'] = key  # Add a new column containing the key
    dfs.append(df)

# Concatenate the dataframes into a single one
result = pd.concat(dfs)

result.to_csv('comp_financials.csv')

result.head()

,key


In [12]:
# Stuff that changes with time (Revenue/Earnings/Employees/ROA/ROE/Free Cash Flow)
# importing the dataframe with the relevant years
#ESG_TR_df = pd.read_csv("tr_esg_df_cleaned.csv")
#
#test_frame = ESG_TR_df.iloc[:30, :]
#
## loop to collect data for all available companies, stored in a dictionary with a df for each isin
#comp_financials_data_dict = {}
#
#for year, isin in zip(test_frame["Year"], test_frame["key"]):
#
#    df = rd.get_history(
#        universe=isin,
#        fields=["TR.RevenueMean", "TR.EarningsMean", "TR.F.EmpAvg", "TR.EPSMean", "TR.ROAMean", "TR.ROEMean",
#        "TR.Volume", "TR.CompanyMarketCap", "TR.FCFMean", "TR.WACC", "TR.EBITDAMean", "TR.OrganicSalesGrowthMean"],
#        interval="1Y",
#        start=str(year) + "-01-01",
#        end=str(year + 1) + "-01-01",
#    )
#    
#    df.index = pd.to_datetime(df.index)
#    df = df.groupby(df.index.year).first()
#
#    dict_key = isin + str(year)
#    comp_financials_data_dict[dict_key] = df
#
## Assume `dfs_dict` is the dictionary of dataframes
#dfs = []
#for key, df in comp_financials_data_dict.items():
#    df['key'] = key  # Add a new column containing the key
#    dfs.append(df)
#
## Concatenate the dataframes into a single one
#comp_financials_test = pd.concat(dfs)
#
## save the new dataframe
#comp_financials_test.to_csv('comp_financials_test.csv')
#
#comp_financials_test.head()

In [13]:
# Stuff that changes with time (Revenue/Earnings/Employees/ROA/ROE/Free Cash Flow)
# importing the dataframe with the relevant years
#ESG_TR_df = pd.read_csv("tr_esg_df_cleaned.csv")
#
#test_frame = ESG_TR_df.iloc[:30, :]
#
## loop to collect data for all available companies, stored in a dictionary with a df for each isin
#comp_financials_data_dict = {}
#
#def financials_collection(year, isin):
#
#
#
#
#    df = rd.get_history(
#        universe=isin,
#        fields=["TR.Revenue", "TR.Earnings", "TR.F.EmpAvg", "TR.EPSMean", "TR.ROAMean", "TR.ROEMean",
#        "TR.Volume", "TR.CompanyMarketCap", "TR.FCFMean", "TR.WACC", "TR.EBITDAMean", "TR.OrganicSalesGrowthMean"],
#        interval="1Y",
#        start=str(year) + "-01-01",
#        end=str(year + 1) + "-01-01",
#    )
#    
#    df.index = pd.to_datetime(df.index)
#    df = df.groupby(df.index.year).first()
#
#    dict_key = isin + str(year)
#
#    comp_financials_data_dict[isin] = df
#
## Assume `dfs_dict` is the dictionary of dataframes
#dfs = []
#for key, df in comp_financials_data_dict.items():
#    df['key'] = key  # Add a new column containing the key
#    dfs.append(df)
#
## Concatenate the dataframes into a single one
#comp_financials_test = pd.concat(dfs)
#
## save the new dataframe
#comp_financials_test.to_csv('comp_financials_test.csv')
#
#comp_financials_test.head()

In [14]:
rd.close_session()